In [1]:
import os

import pandas as pd
import openpyxl
import warnings

import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def filter_by_commodity(data):
    """
    Filters data by commodity and region according to predefined criteria.
    Returns a list of filtered DataFrames.
    """
    filter_conditions = [
        ('World Corn Supply and Use', ['World', 'Major exporters', 'United States', 'Major Exporters']),
        ('World Wheat Supply and Use', ['World', 'Major exporters', 'United States', 'Major Exporters']),
        ('World Soybean Supply and Use', ['World', 'Argentina', 'Brazil', 'United States', 'Major Exporters']),
        ('World Soybean Meal Supply and Use', ['World', 'Major exporters', 'United States', 'Major Exporters']),
        ('World Soybean Oil Supply and Use', ['World', 'Major exporters', 'United States', 'Major Exporters'])
    ]
    
    filtered_data = []
    for title, regions in filter_conditions:
        filtered = data[(data['ReportTitle'] == title) &
                        (data['Region'].isin(regions))]
        filtered_data.append(filtered)
    return filtered_data

def process_by_release_date(data_collection, wasde_data):
    """
    Processes each DataFrame in data_collection, filtering by release date and
    concatenating to wasde_data.
    """
    for data_set in data_collection:
        data_set = data_set[data_set['ProjEstFlag'] == 'Proj.']
        grouped_data = data_set.groupby('ReleaseDate')
        
        for release_date, group in grouped_data:
            filtered_data = group[["ReleaseDate", "Commodity", "Region", "Attribute", "Value", "Unit"]]
            filtered_data = filtered_data.rename(columns={"ReleaseDate": "Report Date"})
            wasde_data = pd.concat([wasde_data, filtered_data], ignore_index=True)
    
    return wasde_data

def process_by_path(path, wasde_data):
    """
    Processes data from a given path and appends it to wasde_data.
    """
    data = pd.read_csv(path, low_memory=False)
    filtered_data = filter_by_commodity(data)
    processed_data = process_by_release_date(filtered_data, wasde_data)
    # processed_data = processed_data.sort_values(by='Report Date', ascending=True)
    return processed_data

def clean_data(data):
    """
    Cleans the data by coercing types and applying a mapping to the 'Attribute' column.
    """
    conversion_dict = {
        'Domestic Feed': 'Feed',
        'Domestic Crush': 'Crush',
        'Domestic Total': 'Total Use',
    }

    data['Report Date'] = pd.to_datetime(data['Report Date']).dt.date
    data['Commodity'] = data['Commodity'].astype(str)
    data['Region'] = data['Region'].astype(str)
    data['Attribute'] = data['Attribute'].astype(str)
    data['Value'] = data['Value'].astype(float)
    data['Unit'] = data['Unit'].astype(str)
    data['Attribute'] = data['Attribute'].replace(conversion_dict)

    data_collection = data.groupby(['Report Date', 'Commodity'])
    sorted_data = pd.concat([group for _, group in data_collection]).sort_values(by=['Report Date', 'Commodity'])

    return sorted_data.reset_index(drop=True)

def process_wasde_data(excel_path, csv_paths, csv_dir, output_path):
    """
    Aggregates WASDE data from multiple sources and saves it to a CSV file.
    """
    # Load and concatenate Excel data
    WASDE_df_dict = pd.read_excel(excel_path, sheet_name=None)
    WASDE_df = pd.concat(WASDE_df_dict.values(), ignore_index=True)
    WASDE_df.rename(columns={"Country": "Region"}, inplace=True)

    # Process CSV paths
    processed_data = pd.DataFrame()
    for path in csv_paths:
        processed_data = process_by_path(path, processed_data)

    # Process files in the CSV directory
    csv_dir_path = Path(csv_dir)
    for file_path in csv_dir_path.glob('*.csv'):
        processed_data = process_by_path(file_path, processed_data)
    
    # Combine all data
    WASDE_df = pd.concat([WASDE_df, processed_data], ignore_index=True)

    # Clean the data and save to CSV
    WASDE_df = clean_data(WASDE_df)
    WASDE_df.to_csv(output_path, index=False)

def aggregate_wasde_data(input_path, output_path):
    # Drop the 'Unit' column
    WASDE_df = pd.read_csv(input_path, low_memory=False)
    WASDE_df = WASDE_df.drop(columns=['Unit'])
    
    # Initialize a list to store the processed rows
    processed_rows = []
    
    # Group by 'Report Date', 'Commodity', and 'Region'
    grouped = WASDE_df.groupby(['Report Date', 'Commodity', 'Region'])
    
    for (report_date, commodity, region), group in grouped:
        # Initialize a dictionary for the row
        row = {
            'Report Date': report_date,
            'Commodity': commodity,
            'Region': region,
            'Beginning Stocks': None,
            'Production': None,
            'Imports': None,
            'Exports': None,
            'Feed/Crush': None,
            'Total Use': None,
            'Ending Stocks': None,
            'STU': None
        }
        
        # Fill the row dictionary based on the 'Attribute' column
        ending_stocks = None
        total_use = None
        stu = None
        for _, record in group.iterrows():
            attribute = record['Attribute']
            value = record['Value']
            
            if attribute == 'Beginning Stocks':
                row['Beginning Stocks'] = value
            elif attribute == 'Production':
                row['Production'] = value
            elif attribute == 'Imports':
                row['Imports'] = value
            elif attribute == 'Exports':
                row['Exports'] = value
            elif attribute in ['Feed', 'Crush']:
                row['Feed/Crush'] = value
            elif attribute in ['Total Use', 'Use, Total']:
                row['Total Use'] = value
                total_use = value
            elif attribute == 'Ending Stocks':
                row['Ending Stocks'] = value
                ending_stocks = value

        if ending_stocks and total_use is not None:
            stu = round(ending_stocks / total_use, 4)
            row['STU'] = stu 

        # Append the row to the list of processed rows
        processed_rows.append(row)
    
    # Create a new DataFrame from the processed rows
    processed_df = pd.DataFrame(processed_rows)
    replacements = {
    'OilSeed, Soybeans': 'Soybeans',
    'Oilseed, Soybean': 'Soybeans',
    'Meal, Soybeans': 'Soybean Meal',
    'Oil, Soybeans': 'Soybean Oil'
    }

    # Replace the entries in the 'Commodity' column
    processed_df['Commodity'] = processed_df['Commodity'].replace(replacements)

    # Save the new DataFrame to a CSV file
    processed_df.to_csv(output_path, index=False)

def process_wasde_soybeans(data_path, output_path):
    data = pd.read_csv(data_path)
    data['Report Date'] = pd.to_datetime(data['Report Date'])
    data['Report Month'] = data['Report Date'].dt.to_period('M')
    rows = []

    grouped = data.groupby('Report Date')

    for report_date, group in grouped:
        new_row = {
            'Report Date': report_date,
            'Report Month': report_date.to_period('M'),
            'STU, US': None,
            'STU, AR': None,
            'STU, BR': None,
            'STU, Corn': None,
            'Production, US': None,
            'Production, AR': None,
            'Production, BR': None,
        }
        
        for _, row in group.iterrows():
            if row['Commodity'] == "Soybeans":
                if row['Region'] == 'United States':
                    new_row['STU, US'] = row['STU']
                    new_row['Production, US'] = row['Production']
                elif row['Region'] == 'Argentina':
                    new_row['STU, AR'] = row['STU']
                    new_row['Production, AR'] = row['Production']
                elif row['Region'] == 'Brazil':
                    new_row['STU, BR'] = row['STU']
                    new_row['Production, BR'] = row['Production']
            
            if row['Commodity'] == 'Corn' and row['Region'] == 'United States':
                new_row['STU, Corn'] = row['STU']
        
        # Append the new row to the list
        rows.append(new_row)

    # Create the new DataFrame from the list of new rows
    processed_data = pd.DataFrame(rows)
    processed_data.to_csv(output_path, index=False)

def append_indicators(data_path, indicator_path, cot_path):
    # Read the main data and indicators data
    df_data = pd.read_csv(data_path)
    df_indicator = pd.read_csv(indicator_path)
    df_cot = pd.read_csv(cot_path)
    
    # Convert 'Report Month' in df_data and 'Date' in df_indicator to period format (YYYY-MM)
    df_data['Report Month'] = pd.to_datetime(df_data['Report Date']).dt.to_period('M')
    df_indicator['Date'] = pd.to_datetime(df_indicator['Date']).dt.to_period('M')
    
    # Merge the main data with the indicators data on 'Report Month' and 'Date'
    merged_df = pd.merge(df_data, df_indicator, how='left', left_on='Report Month', right_on='Date')
    
    # # Convert 'Report Date' in df_cot to period format (YYYY-MM)
    # df_cot['Report Date'] = pd.to_datetime(df_cot['Report_Date']).dt.to_period('M')
    
    # # Group COT data by 'Report Date' and calculate averages for 'MM Position, Net' and 'MM Position, Gross'
    # df_cot['MM Position, Gross'] = df_cot['MM_Long_All'] + df_cot['MM_Short_All']
    # df_cot.rename(columns={'Net_Position':'MM Position, Net'}, inplace=True)
    # grouped_cot = df_cot.groupby('Report Date').agg({
    #     'MM Position, Net': 'mean',
    #     'MM Position, Gross': 'mean'
    # }).reset_index()
    
    # # Merge the previously merged DataFrame with the grouped COT data
    # final_df = pd.merge(merged_df, grouped_cot, how='left', left_on='Report Date', right_on='Report Date')

    merged_df.drop(columns=['Report Month', 'Date'], inplace=True)
    return merged_df

def read_and_process_dates(dates_path):
    """Reads and processes the complete dates CSV."""
    data = pd.read_csv(dates_path).iloc[:-1]
    data = data.rename(columns={'Time': 'Date'})
    data['Date'] = pd.to_datetime(data['Date']).dt.date
    return data.sort_values(by='Date', ascending=True)

def get_sorted_contracts(csv_files, symbols, years):
    """Sorts the CSV files by expiration based on symbols and years."""
    contracts_sorted_by_expiration = []
    for year in years:
        for symbol in symbols:
            contract_name = f"{symbol}{str(year)[-2:]}.csv"
            if contract_name in csv_files:
                contracts_sorted_by_expiration.append(contract_name)
    return contracts_sorted_by_expiration

def create_tuple(row):
    """Creates a tuple of High and Low values or returns an empty string if NaN."""
    return (row['High'], row['Low']) if pd.notna(row['High']) and pd.notna(row['Low']) else ''

def merge_contract_data(csv_dir, contracts_sorted_by_expiration, data):
    """Merges contract data into the main DataFrame."""
    for contract in contracts_sorted_by_expiration:
        contract_name = contract.replace('.csv', '')
        path = os.path.join(csv_dir, contract)
        
        # Read and process the contract CSV
        price_df = pd.read_csv(path).iloc[:-1]
        price_df = price_df.rename(columns={'Time': 'Date'})
        price_df['Date'] = pd.to_datetime(price_df['Date']).dt.date
        price_df['Tuple'] = price_df.apply(create_tuple, axis=1)

        # Merge into the main DataFrame
        data = data.merge(price_df[['Date', 'Tuple']], on='Date', how='left', suffixes=('', f'_{contract_name}'))
        data.rename(columns={'Tuple': f'{contract_name}'}, inplace=True)
    
    return data.fillna('')

def unpack_prices(price_string):
    """Unpacks the High and Low prices from a tuple string."""
    high, low = map(float, price_string.strip('()').split(','))
    return high, low

def determine_contract(month, year):
    """Determines the current and next contract based on the month and year."""
    yr = str(year)[-2:]  # Get the last two digits of the year
    contracts = {
        1: (f'SF{yr}', f'SH{yr}'),
        2: (f'SH{yr}', f'SH{yr}'),
        3: (f'SH{yr}', f'SK{yr}'),
        4: (f'SK{yr}', f'SK{yr}'),
        5: (f'SK{yr}', f'SN{yr}'),
        6: (f'SN{yr}', f'SN{yr}'),
        7: (f'SN{yr}', f'SQ{yr}'),
        8: (f'SQ{yr}', f'SU{yr}'),
        9: (f'SU{yr}', f'SX{yr}'),
        10: (f'SX{yr}', f'SX{yr}'),
        11: (f'SX{yr}', f'SF{str(int(year) + 1)[-2:]}'),  
        12: (f'SF{str(int(year) + 1)[-2:]}', f'SF{str(int(year) + 1)[-2:]}'),
    }
    return contracts.get(month, ('', ''))

def process_continuous_data(dates, wasde_df, historical_df):
    """Processes continuous futures data by selecting the appropriate contracts."""
    continuous_data = []
    index = 0

    for date in dates:
        if index < len(wasde_df):
            wasde_date = wasde_df.iloc[index]['Report Date'].date()
            month, year = wasde_date.month, wasde_date.year
        else:
            break

        current_contract, next_contract = determine_contract(month, year)
        contract_name = current_contract if date < wasde_date else next_contract

        if date >= wasde_date:
            index += 1

        if contract_name and not historical_df.loc[historical_df['Date'].dt.date == date, contract_name].empty:
            pricing_tuple = historical_df.loc[historical_df['Date'].dt.date == date, contract_name].values[0]
            high, low = unpack_prices(pricing_tuple)
            average = (high + low) / 2
            continuous_data.append([date, contract_name, high, low, average])

    return pd.DataFrame(continuous_data, columns=['Date', 'Contract', 'High', 'Low', 'Average'])


### Process and Aggregate WASDE Data

In [3]:
# Define paths
excel_path = '../data/WASDE/World_WASDE_2000-2010.xlsx'
csv_paths = ['../data/WASDE/2010-2020/WASDE_2010-2015.csv', '../data/WASDE/2010-2020/WASDE_2016-2020.csv']
csv_dir = '../data/WASDE/2021-2024'
processed_data_path = '../data/WASDE/processed_WASDE_data.csv'
aggregated_data_path = '../data/WASDE/WASDE.csv'

# Run the aggregation process
process_wasde_data(excel_path, csv_paths, csv_dir, processed_data_path)
aggregate_wasde_data(processed_data_path, aggregated_data_path)

### Process Pricing Data

In [4]:
csv_dir = '../data/soybean_prices/raw'
price_dir = '../data/soybean_prices'
wasde_path = '../data/WASDE/WASDE.csv'
dates_path = os.path.join(price_dir, 'complete_dates.csv')
aggregate_price_path = os.path.join(price_dir, 'aggregate_soybeans.csv')
continuous_price_path = os.path.join(price_dir, 'continuous_soybeans.csv')

# Read and process dates
dates_df = read_and_process_dates(dates_path)
dates = dates_df['Date'].tolist()

# Create the initial DataFrame with dates
data = pd.DataFrame(dates, columns=['Date'])

# Define the base symbols and the range of years
symbols = ["SF", "SH", "SK", "SN", "SQ", "SU", "SX"]
years = list(range(2000, pd.Timestamp.today().year + 1))

# Get the list of CSV files in the directory
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Sort the CSV files by year and symbol order
contracts_sorted_by_expiration = get_sorted_contracts(csv_files, symbols, years)

# Merge the contract data
data = merge_contract_data(csv_dir, contracts_sorted_by_expiration, data)

# Save the final merged DataFrame to a CSV file
data.to_csv(aggregate_price_path, index=False)

# Read the historical and WASDE DataFrames
historical_df = pd.read_csv(aggregate_price_path, parse_dates=['Date'], low_memory=False)
wasde_df = pd.read_csv(wasde_path, parse_dates=['Report Date'])

# Process continuous futures data
continuous_df = process_continuous_data(dates, wasde_df, historical_df)
continuous_df.to_csv(continuous_price_path, index=False)

### Aggregate Macroeconomic Indicators

In [6]:
wasde_path = '../data/WASDE/WASDE.csv'
soybeans_wasde_path = '../data/WASDE/WASDE_soybeans.csv'
macro_path = '../data/macroeconomic_indicators/indicators.csv'
cot_path = '../data/commitment_of_traders/soybeans.csv'
data_output_path = '../data/soybeans_model_input.csv'

process_wasde_soybeans(wasde_path, soybeans_wasde_path)
merged_data = append_indicators(soybeans_wasde_path, macro_path, cot_path)
merged_data.to_csv(data_output_path, index=False)

### Aggregate Price Data

In [7]:
price_path = '../data/soybean_prices/continuous_soybeans.csv'
data_path = '../data/soybeans_model_input.csv'

data_df = pd.read_csv(data_path, low_memory=False)
price_df = pd.read_csv(price_path, low_memory=False)

data_df.rename(columns={'Report Date':'Date'}, inplace=True)
data_df['Date'] = pd.to_datetime(data_df['Date']).dt.date
price_df['Date'] = pd.to_datetime(price_df['Date']).dt.date

report_dates = data_df['Date'].tolist()
report_dates.append(pd.Timestamp.today().date())

price_high_list = []
price_low_list = []
price_average_list = []
price_collections = []

# Iterate through report dates
for i in range(len(report_dates) - 1):
    start_date = report_dates[i]
    end_date = report_dates[i + 1]

    # Filter the price_df for the date range
    group = price_df[(price_df['Date'] >= start_date) & (price_df['Date'] < end_date)]
    
    if not group.empty:
        group = group.head(15)  # Limit to the first 15 rows
        price_array = group['Average'].to_list()

        price_high = group['High'].max()
        price_low = group['Low'].min()
        price_average = group['Average'].mean()
    else:
        # If the group is empty, assign NaN or an appropriate default value
        price_array = []
        price_high = float('nan')
        price_low = float('nan')
        price_average = float('nan')
    
    # Append the results to the respective lists
    price_high_list.append(float(price_high))
    price_low_list.append(float(price_low))
    price_average_list.append(float(price_average))
    price_collections.append(price_array)

# Append the new columns to data_df
data_df['Price_High'] = price_high_list
data_df['Price_Low'] = price_low_list
data_df['Price_Average'] = price_average_list
data_df['Average_Price_Collection'] = price_collections

data_df.to_csv(data_path, index=False)